In [2]:
import osmnx as ox
import pandas as pd
import networkx as nx
import pickle, math
import random
from shapely.geometry import LineString

In [5]:
G = nx.DiGraph()
for u,v,data in blrgraph.edges(data=True):
    w = data['length'] if 'length' in data else 0.0
    if G.has_edge(u,v):
        G[u][v]['weight'] += w
    else:
        G.add_edge(u, v, weight=w)

# print(G.edges(data=True)[5:10])
print(list(G.edges(data=True))[5:10])



KeyboardInterrupt



In [10]:
with open('blr.bin', 'rb') as f:
    Gp = pickle.load(f)

In [ ]:
Gp = ox.graph_from_place("Bengaluru, India", network_type='drive')


In [9]:
with open('test.bin','wb') as f:
    pickle.dump(Gp, f)

NameError: name 'Gp' is not defined

In [ ]:
u,v = tuple(Gp.edges.items())[7]
print(v.keys())
# for u,v in Gp.edges.items():
    # print(v.keys())

dict_keys(['osmid', 'oneway', 'lanes', 'ref', 'name', 'highway', 'maxspeed', 'reversed', 'length'])


In [5]:
def generateIds(leng):
  nodeIds = set()
  while len(nodeIds) < leng:
      nodeId = ''.join(random.choices(('a', 'b', 'c', 'd', 'e', 'f', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'), k=8))
      nodeIds.add(nodeId)
  return list(nodeIds)

In [43]:
a = {2, 3}
a.update([4, 8])
a

{2, 3, 4, 8}

In [24]:
def haversine(pair1, pair2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(math.radians, [pair1[1], pair1[0], pair2[1], pair2[0]])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r * 1000

In [12]:
c = len(Gp.nodes)
for u, v in Gp.edges.items():
    if 'geometry' in v.keys():
        c += len(v['geometry'].wkt[12:-1].split(','))
print(c)

622334


In [47]:
print(c)
nodeIds = generateIds(c)

622334


KeyboardInterrupt: 

In [25]:
G = nx.MultiDiGraph()
temp_dict = dict()
node_index = 0

# for i in Gp.nodes:
#     print(Gp.nodes[i].get('x'))
#     # G.add_node(i)

for u,v in Gp.edges.items():
    if u[0] not in temp_dict.keys():
        nodeId = nodeIds[node_index]; G.add_node(nodeId); temp_dict[u[0]] = nodeId; node_index += 1
    else:
        nodeId = temp_dict[u[0]]
    
    if u[1] not in temp_dict.keys():
        nodeId2 = nodeIds[node_index]; G.add_node(nodeId2); temp_dict[u[1]] = nodeId2; node_index += 1
    else:
        nodeId2 = temp_dict[u[1]]

    G.nodes[nodeId]['lng'] = Gp.nodes[u[0]]['x']
    G.nodes[nodeId]['lat'] = Gp.nodes[u[0]]['y']
    G.nodes[nodeId2]['lng'] = Gp.nodes[u[1]]['x']
    G.nodes[nodeId2]['lat'] = Gp.nodes[u[1]]['y']

    if 'geometry' in v.keys():
        prev_node = nodeId
        for i in v['geometry'].wkt[12:-1].split(','):
            temp_ = i.split()
            
            nodeId3 = nodeIds[node_index]; node_index += 1
            G.add_node(nodeId3)
            
            G.nodes[nodeId3]['lng'] = float(temp_[0])
            G.nodes[nodeId3]['lat'] = float(temp_[1])
            
            G.add_edge(prev_node, nodeId3, dist=haversine((G.nodes[prev_node]['lat'], G.nodes[prev_node]['lng']), (G.nodes[nodeId3]['lat'], G.nodes[nodeId3]['lng'])), oneway=v['oneway'])
            prev_node = nodeId3
        G.add_edge(prev_node, nodeId2, dist=0.0, oneway=v['oneway'])
    else:
        G.add_edge(nodeId, nodeId2, dist=v['length'], oneway=v['oneway'])

print(c == len(G.nodes))

True


In [6]:
with open('data.bin', 'rb') as f:
    nodeIds = pickle.load(f)

In [26]:
with open('graph.bin', 'wb') as f:
    pickle.dump(G, f)

In [7]:
len(nodeIds)

622334

In [27]:
x = random.choice(tuple(G.nodes))
print(x)
print(G.nodes[x])

b3ce6f1a
{'lng': 77.6051769, 'lat': 13.0648177}


In [28]:
x, y = random.choice(tuple(G.edges.items()))
print(G.nodes[x[0]], G.nodes[x[1]])
print(y)

{'lng': 77.5932403, 'lat': 13.0548002} {'lng': 77.593201, 'lat': 13.0560498}
{'dist': 139.01437628867285, 'oneway': True}


In [37]:
for i in y['geometry'].wkt[12:-1].split(','):
    print(i.split())

['77.6011464', '12.9424727']
['77.6010761', '12.9420206']
['77.6009384', '12.9420464']


In [34]:
[(float(x), float(y)) for x, y in [pair.split(" ") for pair in y['geometry'].wkt[12:-1].split(",")]]

ValueError: too many values to unpack (expected 2)

In [ ]:
c = 0;
for u,v in Gp.edges:
    lat1 = Gp.nodes[u].get('y')
    lon1 = Gp.nodes[u].get('x')
    lat2 = Gp.nodes[v].get('y')
    lon2 = Gp.nodes[v].get('x')
    
    f.write(str(lat1) + ',' + str(lon1) + ',' + str(lat2) + ',' + str(lon2) + ',' + str(G[u][v]['weight']) + '\n')

f.close()

ValueError: too many values to unpack (expected 2)

In [ ]:
L = list(G.edges(data=True))[5:10]
f = open('fileS1.txt', 'w')
for t in L :
    line = ' '.join(str(x) for x in t)
    f.write(line + '\n')
f.close()


file1 = open('fileS1.txt', 'r')
print(file1.read())
file1.close()

In [ ]:

print(len(Gp.edges))

404360


In [ ]:
u,v,w = Gp.edges[0]
print(u,v,w)

TypeError: cannot unpack non-iterable int object